In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys

import pandas as pd
import numpy as np

import time

import torch

from tqdm.auto import tqdm

In [2]:
from datasets import load_original_dataset, load_deleted_dataset
from models import CNN

In [3]:
DATA_DIR = 'Datasets/Features/'
BATCH_SIZE = 32
EPOCHS = 5
PERCENTAGES = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99]

In [4]:
sys.path.append(os.path.abspath('./libraries/Unlearnable-Examples/'))
import toolbox

In [5]:
# https://github.com/HanxunH/Unlearnable-Examples/blob/main/perturbation.py#L45

EPSILON = 8
NUM_STEPS = 1
STEP_SIZE = 0.8

In [6]:
# https://github.com/HanxunH/Unlearnable-Examples/blob/main/perturbation.py#L51
# https://github.com/HanxunH/Unlearnable-Examples/blob/main/perturbation.py#L446

noise_generator = toolbox.PerturbationTool(
    epsilon=EPSILON / 255,
    num_steps=NUM_STEPS,
    step_size=STEP_SIZE / 255
)

In [7]:
def add_random_noise(x):
    
    # https://github.com/HanxunH/Unlearnable-Examples/blob/main/dataset.py#L441
    noise = noise_generator.random_noise(noise_shape=x.shape)
    
    # https://github.com/HanxunH/Unlearnable-Examples/blob/main/dataset.py#L448
    noise = noise.mul(255).clamp_(0, 255)
    
    # https://github.com/HanxunH/Unlearnable-Examples/blob/main/perturbation.py#L451
    x = x + noise.to(x.device)
    x = torch.clamp(x, 0, 255)
    
    return x

In [8]:
def fit(model, save_dir, train_set, test_set, forget_set):
    
    os.makedirs(save_dir, exist_ok=True)
    
    train_x, train_y = train_set.tensors
    forget_x, forget_y = forget_set.tensors
    
    forget_x = add_random_noise(forget_x)
    
    train_set = torch.utils.data.TensorDataset(
        torch.concat([train_x, forget_x], dim=0), 
        torch.concat([train_y, forget_y], dim=0), 
    )
    
    optimizer = torch.optim.Adam(model.parameters())
    error = torch.nn.CrossEntropyLoss()
    
    train_loader = torch.utils.data.DataLoader(train_set, batch_size = BATCH_SIZE, shuffle = True, drop_last=True)
    
    train_times = list()
    train_accs, test_accs, forget_accs = list(), list(), list()
    
    for epoch in range(EPOCHS):
        
        # train
        
        accs = list()
        
        train_time = 0
        
        model.train()
        
        start_time = time.time()
        
        for x, y in train_loader:
            optimizer.zero_grad()
            output = model(x.cuda())
            y = y.cuda()
            loss = error(output, y)
            loss.backward()
            optimizer.step()
            
            train_time += time.time() - start_time
            
            predicted = torch.argmax(output.data, dim=-1)
            accs.append((predicted == y).float().mean().detach().cpu().numpy())
            
            start_time = time.time()
            
        train_times.append(train_time)
        train_accs.append(np.mean(accs))
        
        # test
            
        model.eval()
        with torch.no_grad():
            
            x, y = test_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = model(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            test_accs.append(np.mean(accs))
            

            x, y = forget_set.tensors

            for i in range(0, x.shape[0], BATCH_SIZE):

                output = model(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())

            forget_accs.append(np.mean(accs))
        
        # save
        torch.save(model.state_dict(), os.path.join(save_dir, f'{(epoch+1):03d}.pt'))

    return train_times, train_accs, test_accs, forget_accs

In [9]:
results = list()

for percentage in tqdm(PERCENTAGES):
    
    model = CNN().cuda()
        
    model.load_state_dict(torch.load('./weights/init.pt'))
    
    train_set, test_set, forget_set = load_deleted_dataset(DATA_DIR, percentage)
    
    train_times, train_accs, test_accs, forget_accs = fit(model, f'weights/Unlearnable/{percentage}', train_set, test_set, forget_set)
    
    df = pd.DataFrame(zip(train_times, train_accs, test_accs, forget_accs), columns=['train_time', 'train_acc', 'test_acc', 'forget_acc'])
    df['epoch'] = range(1, EPOCHS+1)
    df['percentage'] = percentage
    
    results.append(df)

  0%|          | 0/11 [00:00<?, ?it/s]

In [10]:
results = pd.concat(results).set_index(['percentage', 'epoch'])

results.to_csv('results/Unlearnable.csv')

results

train_time  train_acc  test_acc  forget_acc
percentage epoch                                             
1          1       12.394400   0.854800  0.979133    0.978916
           2       10.792596   0.942600  0.983726    0.983622
           3       10.864631   0.951767  0.986322    0.986069
           4       10.538467   0.954183  0.986022    0.986069
           5       10.659187   0.957767  0.988918    0.989081
10         1       10.545051   0.853067  0.977636    0.977857
           2       10.654274   0.943933  0.981530    0.981911
           3       10.780971   0.952867  0.985423    0.985841
           4       10.817822   0.954033  0.986022    0.985342
           5       10.355096   0.958283  0.987919    0.987774
20         1       10.333398   0.857167  0.973942    0.973973
           2       11.213562   0.941700  0.980731    0.980650
           3       10.539955   0.949483  0.985423    0.984920
           4       11.011899   0.953117  0.985224    0.985329
           5       10.762937   0.956733  0.984724    0.984239
30         1       10.544111   0.837133  0.973842    0.974137
           2       11.545203   0.939650  0.978335    0.978382
           3       10.942711   0.949267  0.984325    0.984518
           4       10.773436   0.953817  0.987320    0.986301
           5       10.393672   0.957033  0.986022    0.984446
40         1       10.614304   0.843233  0.972444    0.972013
           2       10.487042   0.938583  0.979832    0.979098
           3       10.400898   0.946533  0.982827    0.981685
           4       10.895097   0.953767  0.982728    0.983861
           5       10.716718   0.954600  0.985723    0.985801
50         1       10.580053   0.856017  0.978035    0.975719
           2       10.867815   0.938067  0.978734    0.977393
           3       10.512452   0.948017  0.980431    0.981040
           4       10.861669   0.954467  0.986422    0.985362
           5       10.631861   0.955567  0.987420    0.986211
60         1       10.281838   0.862183  0.977436    0.976682
           2       10.508565   0.943183  0.980931    0.981354
           3       10.673586   0.950767  0.982728    0.983223
           4       10.674185   0.955100  0.987520    0.987157
           5       10.916331   0.957500  0.989217    0.988787
70         1       10.543548   0.849433  0.975739    0.975381
           2       10.530491   0.940883  0.981929    0.981338
           3       10.765501   0.951533  0.984225    0.983433
           4       10.790347   0.955167  0.988019    0.987623
           5       10.482557   0.956917  0.986821    0.987066
80         1       10.386471   0.831133  0.976538    0.975662
           2       10.656862   0.941017  0.981030    0.980609
           3       10.508404   0.951750  0.982528    0.982264
           4       10.474500   0.953067  0.985723    0.985418
           5       10.656532   0.955917  0.986022    0.985125
90         1       10.788838   0.834017  0.975339    0.975059
           2       10.536884   0.937917  0.979533    0.978261
           3       10.703653   0.947467  0.984325    0.983118
           4       10.688255   0.952850  0.984924    0.984992
           5       10.419155   0.955633  0.988019    0.987631
99         1       10.636824   0.851500  0.976338    0.972854
           2       10.538013   0.935383  0.980132    0.979450
           3       10.610987   0.949133  0.982328    0.980717
           4       10.454468   0.951583  0.983227    0.984116
           5       10.681539   0.953150  0.986322    0.985484